In [16]:
import pandas as pd
import re
import ipywidgets as widgets


#df = pd.read_csv('games.csv', dtype='object')

rdbuttons = widgets.RadioButtons(
    options=['white', 'black'],
    value='white', # Defaults to 'pineapple'
    layout={'width': 'max-content'}, # If the items' names are long
    description='Play as:',
    disabled=False
)

rdbuttons

RadioButtons(description='Play as:', layout=Layout(width='max-content'), options=('white', 'black'), value='wh…

In [2]:
play_as = rdbuttons.value
print(play_as)

black


In [3]:
#df = df[df['rated'] == 'True']
#df = df[['url', 'rated', 'white.username', 'white.rating', 'white.result','black.username', 'black.rating', 'black.result', 'pgn']]

In [17]:
#df = df.dropna(subset=['pgn'])
#df['white.rating'] = df['white.rating'].astype(int)
#df['black.rating'] = df['black.rating'].astype(int)



if play_as == 'white':
    df = pd.read_csv('white.csv', dtype='object')
    df = df.dropna(subset=['pgn'])
    df = df[df['rated'] == 'True']
    #df = df[(df['black.result'] == 'checkmated') | (df['black.result'] == 'resigned')]
    #df = df[df['white.rating'] < 1300]
else:
    df = pd.read_csv('black.csv', dtype='object')
    df = df.dropna(subset=['pgn'])
    df = df[df['rated'] == 'True']
    #df = df[(df['white.result'] == 'checkmated') | (df['white.result'] == 'resigned')]
    #df = df[df['black.rating'] < 1300]

df = df.sort_values(by=['white.rating', 'black.rating'], ascending=[False, False])


def regmatch(string):
    try:
        s = re.search('1\.\s.*$', string).group(0)
        s = re.sub(r'\{[^{}]*\}', '', s)
    except:
        s = ''
    
    return s



df['pgn'] = df['pgn'].apply(lambda x: regmatch(x))


def next_pgn(string, regex):
    try:
        p = re.search('.*{}'.format(regex), string).group(0)
    except:
        p = ''
    
    return p



tm = 'white'
mv = 1

m = "--"

In [23]:
print(m)
move = input('{} to move:'.format(tm))


if tm == 'white':

    if mv != 1:
        mv2 = ' {}'.format(mv)
    else:
        mv2 = '{}'.format(mv)
    df = df[df['pgn'].str.contains('{}. {}'.format(mv2,move))].reset_index(drop=True)
    
    t = ' {}\.\.\.\s([NBRQK]?)([a-h]?)([x]?)([a-h][1-8])'.format(mv)
    
    df['next_pgn'] = df['pgn'].apply(lambda x: next_pgn(x,t))
    npgn = df.groupby('next_pgn')['pgn'].count().reset_index().sort_values(by='pgn', ascending=False)
    npgn = npgn[npgn['next_pgn'].str.len() > 0].reset_index(drop=True)
    
    st = npgn.next_pgn[0]
    nxt = re.search(t, st).group(0)
    #nxt = nxt.replace(' {}... '.format(mv),'')
    m = 'black to move: {}'.format(nxt)
    print(m)
    
    tm = 'black'
    
else:

    df = df[df['pgn'].str.contains(' {}... {}'.format(mv,move))].reset_index(drop=True)
    mv += 1
    
    t = ' {}\.\s([NBRQK]?)([a-h]?)([x]?)([a-h][1-8])'.format(mv)
    
    df['next_pgn'] = df['pgn'].apply(lambda x: next_pgn(x,t))
    npgn = df.groupby('next_pgn')['pgn'].count().reset_index().sort_values(by='pgn', ascending=False)
    npgn = npgn[npgn['next_pgn'].str.len() > 0].reset_index(drop=True)
    
    st = npgn.next_pgn[0]
    nxt = re.search(t, st).group(0)
    #nxt = nxt.replace(' {}. '.format(mv),'')
    m = 'white to move: {}'.format(nxt)
    print(m)
    
    tm = 'white'
    
    
print(len(df))
print(st)

black to move:  3... Nxe4


black to move: Nex4


KeyError: 0